In [1]:
!pip install mlflow prefect --quiet
!pip install mlflow prefect --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 660.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.6/89.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128

In [4]:
# Install the required packages
!pip install mlflow prefect --quiet

# Import necessary libraries
import mlflow
import mlflow.sklearn
from prefect import flow, task
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Task 1: Load the Iris dataset
@task
def load_data():
    iris = load_iris()
    df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
    df['target'] = iris.target
    X = df.drop('target', axis=1)
    y = df['target']
    return train_test_split(X, y, test_size=0.2, random_state=42)

# Task 2: Train a Random Forest model and log with MLFlow
@task
def train_model(X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
        model.fit(X_train, y_train)

        # Predict and calculate accuracy
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)

        # Log parameters, metrics, and model with MLflow
        mlflow.log_param("n_estimators", 100)
        mlflow.log_param("max_depth", 5)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.sklearn.log_model(model, "model")

        print(f"Model trained with accuracy: {accuracy}")
        return accuracy

# Flow that orchestrates the tasks
@flow(name="MLFlow_Prefect_Pipeline", log_prints=True)
def ml_pipeline():
    X_train, X_test, y_train, y_test = load_data()
    train_model(X_train, X_test, y_train, y_test)

# Run the flow
ml_pipeline()


12:31:56.990 | INFO    | prefect - Starting server on http://127.0.0.1:8730

12:31:57.460 | INFO    | prefect.engine - Created flow run 'yellow-mayfly' for flow 'MLFlow_Prefect_Pipeline'

12:31:57.813 | INFO    | Task run 'load_data-0c1' - Created task run 'load_data-0c1' for task 'load_data'

12:31:57.968 | INFO    | Task run 'load_data-0c1' - Finished in state Completed()

12:31:58.209 | INFO    | Task run 'train_model-e85' - Created task run 'train_model-e85' for task 'train_model'

2024/09/30 12:32:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


12:32:08.678 | INFO    | Task run 'train_model-e85' - Model trained with accuracy: 1.0

12:32:08.703 | INFO    | Task run 'train_model-e85' - Finished in state Completed()

12:32:08.902 | INFO    | Flow run 'yellow-mayfly' - Finished in state Completed()